In [ ]:
import pandas as pd
import json
import os
import pickle
import string

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
input_path='/kaggle/Input/'
output_path='/kaggle/working/'

## Preprocess

### Load data

In [ ]:
folder_path = input_path+'Dataset/SciREX/'
df_scirex=pd.DataFrame()
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path) as f:
        data = [json.loads(line) for line in f]
    df=pd.DataFrame(data)
    df_scirex=pd.concat([df_scirex,df])
df_scirex=df_scirex.reset_index(drop=True)
df_scirex=df_scirex[['sentences','words','ner']]
df_scirex

,sentences,words,ner
0,"[[0, 9], [9, 26], [26, 42], [42, 76], [76, 94]...","[document, :, Deep, Label, Distribution, Learn...","[[2, 6, Method], [7, 9, Task], [9, 12, Method]..."
1,"[[0, 24], [24, 53], [53, 80], [80, 131], [131,...","[In, this, work, we, explore, recent, advances...","[[8, 11, Method], [21, 23, Task], [62, 66, Met..."
2,"[[0, 15], [15, 18], [18, 34], [34, 67], [67, 1...","[Bridging, Saliency, Detection, to, Weakly, Su...","[[0, 3, Task], [4, 8, Task], [10, 15, Method],..."
3,"[[0, 11], [11, 50], [50, 97], [97, 117], [117,...","[document, :, The, IBM, 2016, English, Convers...","[[3, 11, Task], [16, 21, Method], [24, 27, Met..."
4,"[[0, 9], [9, 34], [34, 52], [52, 106], [106, 1...","[document, :, Rethinking, the, Inception, Arch...","[[4, 6, Method], [7, 9, Task], [9, 11, Method]..."
...,...,...,...
433,"[[0, 8], [8, 24], [24, 53], [53, 83], [83, 102...","[document, :, Invertible, Conditional, GANs, f...","[[2, 5, Method], [6, 8, Task], [8, 11, Method]..."
434,"[[0, 8], [8, 11], [11, 30], [30, 51], [51, 80]...","[Asymmetric, Tri, -, training, for, Unsupervis...","[[0, 4, Method], [5, 8, Task], [11, 15, Method..."
435,"[[0, 8], [8, 11], [11, 30], [30, 45], [45, 66]...","[A, Convolutional, Encoder, Model, for, Neural...","[[1, 4, Method], [5, 8, Task], [15, 18, Task],..."
436,"[[0, 13], [13, 46], [46, 67], [67, 103], [103,...","[document, :, Learning, to, Compare, :, Relati...","[[2, 3, Task], [6, 8, Method], [9, 13, Task], ..."


### Reformat

In [ ]:
df_scirex['new-sentences'] = df_scirex.apply(lambda row: [row['words'][start:end] for start, end in row['sentences']], axis=1)
df_scirex

,sentences,words,ner,new-sentences
0,"[[0, 9], [9, 26], [26, 42], [42, 76], [76, 94]...","[document, :, Deep, Label, Distribution, Learn...","[[2, 6, Method], [7, 9, Task], [9, 12, Method]...","[[document, :, Deep, Label, Distribution, Lear..."
1,"[[0, 24], [24, 53], [53, 80], [80, 131], [131,...","[In, this, work, we, explore, recent, advances...","[[8, 11, Method], [21, 23, Task], [62, 66, Met...","[[In, this, work, we, explore, recent, advance..."
2,"[[0, 15], [15, 18], [18, 34], [34, 67], [67, 1...","[Bridging, Saliency, Detection, to, Weakly, Su...","[[0, 3, Task], [4, 8, Task], [10, 15, Method],...","[[Bridging, Saliency, Detection, to, Weakly, S..."
3,"[[0, 11], [11, 50], [50, 97], [97, 117], [117,...","[document, :, The, IBM, 2016, English, Convers...","[[3, 11, Task], [16, 21, Method], [24, 27, Met...","[[document, :, The, IBM, 2016, English, Conver..."
4,"[[0, 9], [9, 34], [34, 52], [52, 106], [106, 1...","[document, :, Rethinking, the, Inception, Arch...","[[4, 6, Method], [7, 9, Task], [9, 11, Method]...","[[document, :, Rethinking, the, Inception, Arc..."
...,...,...,...,...
433,"[[0, 8], [8, 24], [24, 53], [53, 83], [83, 102...","[document, :, Invertible, Conditional, GANs, f...","[[2, 5, Method], [6, 8, Task], [8, 11, Method]...","[[document, :, Invertible, Conditional, GANs, ..."
434,"[[0, 8], [8, 11], [11, 30], [30, 51], [51, 80]...","[Asymmetric, Tri, -, training, for, Unsupervis...","[[0, 4, Method], [5, 8, Task], [11, 15, Method...","[[Asymmetric, Tri, -, training, for, Unsupervi..."
435,"[[0, 8], [8, 11], [11, 30], [30, 45], [45, 66]...","[A, Convolutional, Encoder, Model, for, Neural...","[[1, 4, Method], [5, 8, Task], [15, 18, Task],...","[[A, Convolutional, Encoder, Model, for, Neura..."
436,"[[0, 13], [13, 46], [46, 67], [67, 103], [103,...","[document, :, Learning, to, Compare, :, Relati...","[[2, 3, Task], [6, 8, Method], [9, 13, Task], ...","[[document, :, Learning, to, Compare, :, Relat..."


In [ ]:
df_scirex['new-ner'] = pd.Series(dtype='object')
for i in df_scirex.index:
    ner_list=[]
    flag1=0
    flag2=0
    for j in range(len(df_scirex.sentences[i])):
        end=df_scirex.sentences[i][j][1]
        for k in range(flag2,len(df_scirex.ner[i])):
          pos1=df_scirex.ner[i][k][0]
          if int(pos1)>=int(end):
            flag1=flag2
            flag2=k
            break
        new_ner=df_scirex.ner[i][flag1:flag2]
        ner_list.append(new_ner)
    df_scirex['new-ner'][i]=ner_list
df_scirex

,sentences,words,ner,new-sentences,new-ner
0,"[[0, 9], [9, 26], [26, 42], [42, 76], [76, 94]...","[document, :, Deep, Label, Distribution, Learn...","[[2, 6, Method], [7, 9, Task], [9, 12, Method]...","[[document, :, Deep, Label, Distribution, Lear...","[[[2, 6, Method], [7, 9, Task]], [[9, 12, Meth..."
1,"[[0, 24], [24, 53], [53, 80], [80, 131], [131,...","[In, this, work, we, explore, recent, advances...","[[8, 11, Method], [21, 23, Task], [62, 66, Met...","[[In, this, work, we, explore, recent, advance...","[[[8, 11, Method], [21, 23, Task]], [], [[62, ..."
2,"[[0, 15], [15, 18], [18, 34], [34, 67], [67, 1...","[Bridging, Saliency, Detection, to, Weakly, Su...","[[0, 3, Task], [4, 8, Task], [10, 15, Method],...","[[Bridging, Saliency, Detection, to, Weakly, S...","[[[0, 3, Task], [4, 8, Task], [10, 15, Method]..."
3,"[[0, 11], [11, 50], [50, 97], [97, 117], [117,...","[document, :, The, IBM, 2016, English, Convers...","[[3, 11, Task], [16, 21, Method], [24, 27, Met...","[[document, :, The, IBM, 2016, English, Conver...","[[[3, 11, Task]], [[16, 21, Method], [24, 27, ..."
4,"[[0, 9], [9, 34], [34, 52], [52, 106], [106, 1...","[document, :, Rethinking, the, Inception, Arch...","[[4, 6, Method], [7, 9, Task], [9, 11, Method]...","[[document, :, Rethinking, the, Inception, Arc...","[[[4, 6, Method], [7, 9, Task]], [[9, 11, Meth..."
...,...,...,...,...,...
433,"[[0, 8], [8, 24], [24, 53], [53, 83], [83, 102...","[document, :, Invertible, Conditional, GANs, f...","[[2, 5, Method], [6, 8, Task], [8, 11, Method]...","[[document, :, Invertible, Conditional, GANs, ...","[[[2, 5, Method], [6, 8, Task]], [[8, 11, Meth..."
434,"[[0, 8], [8, 11], [11, 30], [30, 51], [51, 80]...","[Asymmetric, Tri, -, training, for, Unsupervis...","[[0, 4, Method], [5, 8, Task], [11, 15, Method...","[[Asymmetric, Tri, -, training, for, Unsupervi...","[[[0, 4, Method], [5, 8, Task]], [], [[11, 15,..."
435,"[[0, 8], [8, 11], [11, 30], [30, 45], [45, 66]...","[A, Convolutional, Encoder, Model, for, Neural...","[[1, 4, Method], [5, 8, Task], [15, 18, Task],...","[[A, Convolutional, Encoder, Model, for, Neura...","[[[1, 4, Method], [5, 8, Task]], [], [[15, 18,..."
436,"[[0, 13], [13, 46], [46, 67], [67, 103], [103,...","[document, :, Learning, to, Compare, :, Relati...","[[2, 3, Task], [6, 8, Method], [9, 13, Task], ...","[[document, :, Learning, to, Compare, :, Relat...","[[[2, 3, Task], [6, 8, Method], [9, 13, Task]]..."


In [ ]:
df_scirex =df_scirex[['new-sentences','new-ner']].rename(columns={'new-sentences':'sentences','new-ner':'ner'})
df_scirex

,sentences,ner
0,"[[document, :, Deep, Label, Distribution, Lear...","[[[2, 6, Method], [7, 9, Task]], [[9, 12, Meth..."
1,"[[In, this, work, we, explore, recent, advance...","[[[8, 11, Method], [21, 23, Task]], [], [[62, ..."
2,"[[Bridging, Saliency, Detection, to, Weakly, S...","[[[0, 3, Task], [4, 8, Task], [10, 15, Method]..."
3,"[[document, :, The, IBM, 2016, English, Conver...","[[[3, 11, Task]], [[16, 21, Method], [24, 27, ..."
4,"[[document, :, Rethinking, the, Inception, Arc...","[[[4, 6, Method], [7, 9, Task]], [[9, 11, Meth..."
...,...,...
433,"[[document, :, Invertible, Conditional, GANs, ...","[[[2, 5, Method], [6, 8, Task]], [[8, 11, Meth..."
434,"[[Asymmetric, Tri, -, training, for, Unsupervi...","[[[0, 4, Method], [5, 8, Task]], [], [[11, 15,..."
435,"[[A, Convolutional, Encoder, Model, for, Neura...","[[[1, 4, Method], [5, 8, Task]], [], [[15, 18,..."
436,"[[document, :, Learning, to, Compare, :, Relat...","[[[2, 3, Task], [6, 8, Method], [9, 13, Task]]..."


In [ ]:
stop_words=set(stopwords.words("english"))

In [ ]:
df_scirex['tags']=pd.Series(dtype='object')
for i in df_scirex.ner.index:
    tag_list=[]
    length1=0
    for j,sentence in enumerate(df_scirex.ner[i]):
        length = len(df_scirex.sentences[i][j])
        if j > 0:
            length1 += len(df_scirex.sentences[i][j-1])
        flag_task_list = []
        flag_method_list = []
        flag_material_list = []
        for item in sentence:
            start = int(item[0])
            end = int(item[1])
            for pos in range(start, end+1):
                if item[2] == 'Task':
                    flag_task_list.append(pos - length1)
                elif item[2] == 'Method':
                    flag_method_list.append(pos - length1)
                elif item[2] == 'Material':
                    flag_material_list.append(pos - length1)
        tags=[]
        for k in range(length):
            if k in flag_task_list:
                if k > 0 and (tags[k-1] =='B-TSK' or tags[k-1] =='I-TSK'):
                    tag='I-TSK'
                else:
                    tag='B-TSK'
            elif k in flag_method_list:
                if k > 0 and (tags[k-1] =='B-MTD' or tags[k-1] =='I-MTD'):
                    tag='I-MTD'
                else:
                    tag='B-MTD'
            elif k in flag_material_list:
                if k > 0 and (tags[k-1] =='B-DST' or tags[k-1] =='I-DST'):
                    tag='I-DST'
                else:
                    tag='B-DST'
            else:
                tag='O'
            # re-label the symbols and stop words to 'O'
            if df_scirex.sentences[i][j][k].lower() in stop_words:
                tag='O'
            tags.append(tag)
        tag_list.append(tags)
    df_scirex['tags'][i]=tag_list
df_scirex

,sentences,ner,tags
0,"[[document, :, Deep, Label, Distribution, Lear...","[[[2, 6, Method], [7, 9, Task]], [[9, 12, Meth...","[[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, ..."
1,"[[In, this, work, we, explore, recent, advance...","[[[8, 11, Method], [21, 23, Task]], [], [[62, ...","[[O, O, O, O, O, O, O, O, B-MTD, I-MTD, I-MTD,..."
2,"[[Bridging, Saliency, Detection, to, Weakly, S...","[[[0, 3, Task], [4, 8, Task], [10, 15, Method]...","[[B-TSK, I-TSK, I-TSK, O, B-TSK, I-TSK, I-TSK,..."
3,"[[document, :, The, IBM, 2016, English, Conver...","[[[3, 11, Task]], [[16, 21, Method], [24, 27, ...","[[O, O, O, B-TSK, I-TSK, I-TSK, I-TSK, I-TSK, ..."
4,"[[document, :, Rethinking, the, Inception, Arc...","[[[4, 6, Method], [7, 9, Task]], [[9, 11, Meth...","[[O, O, O, O, B-MTD, I-MTD, O, B-TSK, I-TSK], ..."
...,...,...,...
433,"[[document, :, Invertible, Conditional, GANs, ...","[[[2, 5, Method], [6, 8, Task]], [[8, 11, Meth...","[[O, O, B-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK],..."
434,"[[Asymmetric, Tri, -, training, for, Unsupervi...","[[[0, 4, Method], [5, 8, Task]], [], [[11, 15,...","[[B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK,..."
435,"[[A, Convolutional, Encoder, Model, for, Neura...","[[[1, 4, Method], [5, 8, Task]], [], [[15, 18,...","[[O, B-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK, I-T..."
436,"[[document, :, Learning, to, Compare, :, Relat...","[[[2, 3, Task], [6, 8, Method], [9, 13, Task]]...","[[O, O, B-TSK, O, O, O, B-MTD, I-MTD, O, O, B-..."


In [ ]:
df_scirex=df_scirex[['sentences','tags']]
df_scirex

,sentences,tags
0,"[[document, :, Deep, Label, Distribution, Lear...","[[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, ..."
1,"[[In, this, work, we, explore, recent, advance...","[[O, O, O, O, O, O, O, O, B-MTD, I-MTD, I-MTD,..."
2,"[[Bridging, Saliency, Detection, to, Weakly, S...","[[B-TSK, I-TSK, I-TSK, O, B-TSK, I-TSK, I-TSK,..."
3,"[[document, :, The, IBM, 2016, English, Conver...","[[O, O, O, B-TSK, I-TSK, I-TSK, I-TSK, I-TSK, ..."
4,"[[document, :, Rethinking, the, Inception, Arc...","[[O, O, O, O, B-MTD, I-MTD, O, B-TSK, I-TSK], ..."
...,...,...
433,"[[document, :, Invertible, Conditional, GANs, ...","[[O, O, B-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK],..."
434,"[[Asymmetric, Tri, -, training, for, Unsupervi...","[[B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK,..."
435,"[[A, Convolutional, Encoder, Model, for, Neura...","[[O, B-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK, I-T..."
436,"[[document, :, Learning, to, Compare, :, Relat...","[[O, O, B-TSK, O, O, O, B-MTD, I-MTD, O, O, B-..."


In [ ]:
with open(output_path+'Data/df_scirex.pkl', 'wb') as file:
    pickle.dump(df_scirex, file)

## Statistics overview

In [ ]:
length_sentences=[]
length_tokens=[]
for item in df_scirex.sentences:
  length=0
  length_sentences.append(len(item))
  for sentence in item:
    length+=len(sentence)
  length_tokens.append(length)

In [ ]:
df_length_sen=pd.DataFrame(length_sentences)
df_length_tok=pd.DataFrame(length_tokens)

In [ ]:
df_length_sen.describe()

,0
count,438.000000
mean,274.958904
std,98.758324
min,75.000000
25%,214.000000
50%,261.500000
75%,319.750000
max,736.000000


In [ ]:
df_length_tok.describe()

,0
count,438.000000
mean,5678.289954
std,1909.883935
min,1204.000000
25%,4691.000000
50%,5441.000000
75%,6413.000000
max,13731.000000
